# 🌐 ADK + MCP: Connect your Agents to the Universe of Tools!
## Model Context Protocol - Class 4

---

## 📋 Index
1. [Initial Setup](#initial-setup)
2. [What is the Model Context Protocol (MCP)?](#what-is-mcp)
3. [ADK and MCP Integration Patterns](#integration-patterns)
4. [Example 1: ADK with Filesystem MCP Server](#filesystem-example)
5. [Example 2: ADK with Third-party MCP Server](#third-party-example)
6. [Creating an MCP Server with ADK](#create-server)
7. [Key Considerations](#considerations)
8. [Exercises and Resources](#exercises)

## 🎯 Introduction

In this class we will learn to integrate the **Model Context Protocol (MCP)** with our ADK agents. MCP is an open standard that allows LLMs to communicate with external applications in a standardized way.

### What you will learn:
- 🔌 Connect ADK agents to existing MCP servers
- 🛠️ Use MCP tools as if they were native ADK tools
- 🌐 Expose ADK tools through MCP servers
- 🚀 Exponentially expand your agents' capabilities

## 1. Initial Setup {#initial-setup}

### Installing dependencies

In [ ]:
# Install Google ADK and MCP
!pip install -qU google-adk==1.4.2 mcp==1.9.4 python-dotenv

# Install Node.js on Colab (needed to run MCP servers)
!apt-get update && apt-get install -y nodejs npm

# Verify installations
!node --version
!npm --version
!npx --version

print("\n✅ Dependencies installed correctly!")

In [ ]:
# Necessary imports
import os
import json
from typing import List, Dict, Optional
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioServerParameters
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from getpass import getpass

In [ ]:
# Request API Key securely
if 'GOOGLE_API_KEY' not in os.environ:
    print("🔑 Please enter your Google API Key:")
    api_key = getpass("API Key: ")
    os.environ['GOOGLE_API_KEY'] = api_key
    os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
    print("\n✅ API Key configured correctly")
else:
    print("✅ API Key already configured")

# Verify that variables are configured
print(f"\n📋 Configured environment variables:")
print(f"   - GOOGLE_API_KEY: {'✓' if os.environ.get('GOOGLE_API_KEY') else '✗'}")
print(f"   - GOOGLE_GENAI_USE_VERTEXAI: {os.environ.get('GOOGLE_GENAI_USE_VERTEXAI', 'Not configured')}")

print("✅ ADK installed and configured correctly!")

In [ ]:
from dotenv import load_dotenv
# Load environment variables from .env if it exists
load_dotenv(override=True)

### Our inference function

In [ ]:
async def call_agent_async(query: str, runner, user_id, session_id):
    """Sends a query to the agent and prints the final response."""
    print(f"\n>>> User query: {query}")

    # Prepare the user message in ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "The agent did not produce a final response." # Default value

    # Key concept: run_async executes the agent's logic and generates events.
    # We iterate through events to find the final response.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # You can uncomment the line below to see *all* events during execution
        # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

        # Key concept: is_final_response() marks the message that concludes the turn.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Assume the text response is in the first part
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Handle possible errors/escalations
                final_response_text = f"The agent escalated: {event.error_message or 'No specific message.'}"
            # Add more validations here if needed (e.g., specific error codes)
            break # Stop processing events once final response is found

    print(f"<<< Agent response: {final_response_text}")

## 2. What is the Model Context Protocol (MCP)? {#what-is-mcp}

The **Model Context Protocol (MCP)** is an open standard that standardizes how LLMs communicate with external applications.

### Key Concepts:

- **🔌 Universal Connection**: MCP acts as a "common language" between LLMs and external services
- **📡 Client-Server Architecture**:
  - **MCP Server**: Exposes resources, prompts and tools
  - **MCP Client**: Consumes these capabilities (like our ADK agents)
- **🛠️ Standardized Tools**: Tools are described with a common schema

In [ ]:
# Conceptual visualization of MCP
print("""
🤖 ADK AGENT (MCP Client)
        ↕️ [MCP Protocol]
📦 MCP SERVERS
    ├── 📁 File System
    ├── 🗺️ Google Maps
    ├── 💾 Databases
    ├── 🌐 Web APIs
    └── 🔧 Custom Tools

✨ One protocol, infinite possibilities!
""")

## 3. ADK and MCP Integration Patterns {#integration-patterns}

### Two main patterns:

#### 1️⃣ **ADK as MCP Client** (most common)
- Your ADK agent uses tools from existing MCP servers
- We use `MCPToolset` to connect

#### 2️⃣ **ADK as MCP Provider**
- We expose ADK tools through an MCP server
- Other systems can use our tools

## 4. Example 1: ADK with Filesystem MCP Server {#filesystem-example}

### 📁 We will connect an ADK agent to an MCP server that provides file operations

In [ ]:
# Create folder and test files
import os

# Create working directory
WORK_DIR = "/Users/alarcon7a/GIT/google-adk-course/sources_en/Class 4 - MCP/test_folder"
os.makedirs(WORK_DIR, exist_ok=True)

# Create example files
files_to_create = {
    "readme.txt": "Welcome to the MCP with ADK tutorial!\nThis is an example file.",
    "data.json": json.dumps({"name": "ADK", "version": "1.0", "features": ["agents", "tools", "mcp"]}, indent=2),
    "shopping_list.txt": "- Milk\n- Bread\n- Eggs\n- Coffee\n- Fruits",
    "notes.md": "# Course Notes\n\n## MCP\n- Model Context Protocol\n- ADK Integration\n- Practical Examples"
}

for filename, content in files_to_create.items():
    with open(os.path.join(WORK_DIR, filename), "w") as f:
        f.write(content)

print(f"✅ Working directory created: {WORK_DIR}")
print("\n📁 Files created:")
for filename in os.listdir(WORK_DIR):
    print(f"  - {filename}")

In [ ]:
from google.adk.tools.mcp_tool.mcp_toolset import StdioConnectionParams

In [ ]:
# Create agent with MCPToolset for filesystem
filesystem_agent = LlmAgent(
    model='gemini-2.5-flash',
    name='filesystem_assistant',
    description='Assistant for file management using MCP',
    instruction=(
        "You are an expert file management assistant. "
        "You can list files, read their content and help the user "
        "organize their information. You work with the directory: " + WORK_DIR
    ),
    tools=[
        MCPToolset(
            connection_params=StdioConnectionParams(
            server_params=StdioServerParameters(
                command='npx',
                args=[
                    "-y",  # Argument for npx to auto-confirm install
                    "@modelcontextprotocol/server-filesystem",
                    # IMPORTANT: This MUST be an ABSOLUTE path to a folder the
                    # npx process can access.
                    # Replace with a valid absolute path on your system.
                    os.path.abspath(WORK_DIR),
                ],
            ),
            timeout=60
            ),
        )
    ],
    generate_content_config=types.GenerateContentConfig(
        temperature=0.1,
        max_output_tokens=500
    )
)

print("✅ Filesystem agent created with MCPToolset")
print("\n🔧 The agent can:")
print("  - List files in the directory")
print("  - Read file contents")
print("  - Navigate folder structure")

### Test the filesystem agent

In [ ]:
# Key concept: SessionService stores conversation history and state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants to identify the interaction context
APP_NAME = "mcp_filesystem_tutorial"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will occur
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
# Runner: This is the main component that manages interaction with the agent.
runner = Runner(agent=filesystem_agent,
                app_name=APP_NAME,
                session_service=session_service)


In [ ]:
await call_agent_async(
    query="What files are in the test_folder directory?",
    runner=runner,
    user_id=USER_ID,
    session_id=SESSION_ID
)

In [ ]:
await call_agent_async(
    query="Can you read the content of readme.txt?",
    runner=runner,
    user_id=USER_ID,
    session_id=SESSION_ID
)

In [ ]:
await call_agent_async(
    query="Create a new file called 'translated_readme.txt' with the text from readme.txt but translate the content to Spanish.",
    runner=runner,
    user_id=USER_ID,
    session_id=SESSION_ID
)

## 5. Example 2: ADK with Third-party MCP Server {#third-party-example}

🗺️ We will connect an agent to Google Maps through MCP (In the [MCP_Maps](MCP_Maps) folder)

🐦 We will connect an agent to Twitter through MCP (In the [MCP_Twitter](MCP_Twitter) folder)

---

## 6. Creating an MCP Server with ADK {#create-server}

### 🛠️ Now we will expose an ADK tool through MCP

### 6.1 What is MCP?
Model Context Protocol (MCP) is a standardized protocol for LLMs to communicate with external applications. It works with a client-server architecture where:

- **MCP Server**: Exposes tools/functions (your e-commerce logic)
- **MCP Client**: Consumes these tools (the ADK agent)

### 6.2. Main Server Components

#### These imports bring:

- `mcp_types`: MCP protocol data types
- `Server`: Base class for creating an MCP server
- `InitializationOptions`: Initialization configuration
- `stdio`: Standard input/output communication (stdin/stdout)

---

In [ ]:
from mcp import types as mcp_types
from mcp.server.lowlevel import Server
from mcp.server.models import InitializationOptions
import mcp.server.stdio

### 6.3. Communication Flow

**ADK Client ←→ MCPToolset ←→ [stdin/stdout] ←→ MCP Server**

The MCP server communicates through:

- `stdin`: Receives commands from client
- `stdout`: Sends responses to client
- **JSON-RPC Protocol**: Structured messages

---

### 6.4 Main Handlers

```python
@app.list_tools()
async def list_mcp_tools() -> list[mcp_types.Tool]:
```

**Purpose**: When a client connects, it first asks "what tools do you have available?"

**Process**:

- Client sends: `list_tools request`
- Server responds with an array of `mcp_types.Tool`, including:
  - `name`: Unique identifier
  - `description`: What the tool does
  - `inputSchema`: JSON schema of expected parameters

```python
@app.call_tool()
async def call_mcp_tool(name: str, arguments: dict) -> list[mcp_types.Content]:
```

**Purpose**: Executes a specific tool when the client requests it.

**Process**:

- Client sends: `call_tool` with `name` and `arguments`
- Server executes corresponding logic
- Server responds with `mcp_types.Content` (result)

---

### 6.5 Complete Execution Flow

```python
async def run_mcp_stdio_server():
    async with mcp.server.stdio.stdio_server() as (read_stream, write_stream):
        await app.run(
            read_stream,
            write_stream,
            InitializationOptions(...)
        )
```

**Initialization**:

- Server is created with `Server("ecommerce-mcp-server")`
- Handlers are configured (`list_tools`, `call_tool`)

**Handshake**:

- Client connects
- Server sends its capabilities
- Client requests tool list

**Operation Cycle**:

```
Client: "I want to search for laptop"
↓
MCPToolset: call_tool("search_product", {"product_name": "laptop"})
↓
MCP Server: Execute search
↓
MCP Server: Return result as JSON
↓
Client: Receive and display result
```

---

### 6.6 Architecture Diagram

```
┌─────────────────┐
│      User       │
└────────┬────────┘
         │
┌────────▼────────┐
│   ADK Agent     │
│  (MCP Client)   │
└────────┬────────┘
         │
┌────────▼────────┐
│   MCPToolset    │ ← Manages connection
└────────┬────────┘
         │ JSON-RPC over stdio
┌────────▼────────┐
│   MCP Server    │
│ ┌─────────────┐ │
│ │ list_tools  │ │ ← Exposes tools
│ └─────────────┘ │
│ ┌─────────────┐ │
│ │ call_tool   │ │ ← Executes logic
│ └─────────────┘ │
│ ┌─────────────┐ │
│ │    State    │ │ ← Cart, products
│ └─────────────┘ │
└─────────────────┘
```

> This architecture allows your e-commerce logic to be completely independent of the ADK agent, making it more modular and reusable.

### 6.7 Hands On

Let's go to the ecommerce_mcp folder in this class.

There you will find [ecommerce_mcp_server.py](MCP_Ecommerce/ecommerce_mcp_server.py) script where we deploy our ecommerce from the previous class as an MCP server.

And our agent that consumes this custom server as a client [agent.py](MCP_Ecommerce/agent.py)

---

## 7. Key Considerations {#considerations}

### 🔑 Important points when working with ADK and MCP:

### 1. Protocol vs Library

* **MCP**: Is a protocol specification
* **ADK**: Is a Python library
* **MCPToolset**: Implements the MCP client in ADK

### 2. ADK Tools vs MCP

* **ADK Tools**: Python objects (Function calling, Built-in tools)
* **MCP Tools**: Capabilities exposed according to MCP schema
* **Conversion**: `MCPToolset` converts automatically

### 3. Asynchronous Nature

* **ADK**: Based on `asyncio`
* **MCP**: Also asynchronous
* **Implication**: Use `async/await` correctly

### 4. Connection Management

* **MCPToolset**: Manages lifecycle
* **Sessions**: MCP maintains state
* **Cleanup**: Important to close connections

### 5. Paths and Permissions

* **Absolute paths**: Always use complete paths
* **Permissions**: MCP server needs access
* **Security**: Limit tool scope

---

### 🌟 Best Practices


### 1. 🔒 Security

* ✅ Validate all inputs in MCP tools
* ✅ Limit access to sensitive resources
* ✅ Use environment variables for API keys

### 2. 🚀 Performance

* ✅ Reuse connections when possible
* ✅ Implement appropriate timeouts
* ✅ Cache results when it makes sense

### 3. 🛠️ Development

* ✅ Test tools individually first
* ✅ Use detailed logs for debugging
* ✅ Document tool schemas clearly

### 4. 📦 Deployment

* ✅ Containerize MCP servers
* ✅ Monitor active connections
* ✅ Implement health checks

### 5. 🔄 Maintenance

* ✅ Version protocols and tools
* ✅ Document API changes
* ✅ Maintain backward compatibility

---

## 8. Exercises and Resources {#exercises}

### 🎯 Practical Exercises


Create a tool that:

1. Takes a **URL** as input
2. Extracts the **title** and **meta description** from the page
3. Returns a **structured summary** in `dict` format
4. Expose it through an **MCP server**


##### 🧰 Starter Template

```python
@tool
def analyze_web_page(url: str) -> dict:
    """
    TODO: Implement web page analysis
    """
    # Your code here
    pass
```

### 📚 Additional Resources


#### Official Documentation

* **[ADK Docs](https://github.com/google/adk-python)**
* **[MCP Spec](https://modelcontextprotocol.io)**
* **[MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)**
* **[MCP Servers](https://github.com/modelcontextprotocol/servers)**

#### Available MCP Servers

* **Filesystem**: `@modelcontextprotocol/server-filesystem`
* **Google Maps**: `@modelcontextprotocol/server-google-maps`
* **GitHub**: `@modelcontextprotocol/server-github`
* **Slack**: `@modelcontextprotocol/server-slack`

#### Useful ADK Tools

* **load_web_page**: Load web content
* **google_search**: Google search
* **code_execution**: Execute Python code

## 🎉 Congratulations!

You have learned to integrate the Model Context Protocol with your ADK agents. Now you can:

- ✅ Connect ADK agents to existing MCP servers
- ✅ Use `MCPToolset` to consume MCP tools
- ✅ Create MCP servers to expose ADK tools
- ✅ Build more flexible and powerful agent architectures

### 🚀 Next Steps


1. 🔧 Explore more community MCP servers
2. 🛠️ Create your own specialized MCP servers
3. 🌐 Integrate external services via MCP
4. 🏗️ Build multi-agent architectures with MCP
5. 📦 Deploy MCP servers in production

> The MCP ecosystem is constantly growing!
> Stay updated and share your creations with the community.

---

**Created with ❤️ for the ADK developer community**

📝 **Note**: This notebook is part of the Google ADK tutorial series. For more content, visit the official resources.